## plot the result to inspect

In [ ]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import geopandas as gpd
import pandas as pd
import plotly.express as px
import json
import numpy as np
import os
from shapely.geometry import Point
from shapely import wkt
# Process the 'reeds_ba_list' column to expand the sets into individual rows, keeping 'state' intact
from ast import literal_eval

def createshapefile():
    # Get the current directory where your script is running
    current_directory = os.getcwd()

    # Construct the path to your resources folder dynamically
    resources_path = os.path.join(current_directory, 'resources')

    # Now, build the full path to your CSV files
    polygons_csv_path = os.path.join(resources_path, 'US_CAN_MEX_PCA_polygons.csv')
    state_to_ba_csv_path = os.path.join(resources_path, 'state_to_ba_mapping.csv')
    # Finally, use pandas to read the CSV files
    polygons_df = pd.read_csv(polygons_csv_path)
    state_to_ba_df = pd.read_csv(state_to_ba_csv_path)
    # Filter polygons
    polygons_df = polygons_df[polygons_df['rb'].isin([f'p{i}' for i in range(1, 135)])]

    # Since the instructions for processing the state mapping are a bit unclear,
    # Convert the string representation of sets in 'reeds_ba_list' to actual sets
    state_to_ba_df['reeds_ba_list'] = state_to_ba_df['reeds_ba_list'].apply(literal_eval)

    # Explode the sets into separate rows
    exploded_df = state_to_ba_df.explode('reeds_ba_list')[['state','reeds_ba_list']]
    polygons_df_filtered = polygons_df[polygons_df['rb'].isin([f'p{i}' for i in range(1, 135)])]
    polygons_gdf = gpd.GeoDataFrame(polygons_df_filtered, geometry=gpd.GeoSeries.from_wkt(polygons_df_filtered['WKT']))

    # Step 2: Merge exploded_df with polygons_gdf on the 'reeds_ba_list' and 'rb' columns
    merged_gdf = polygons_gdf.merge(exploded_df, left_on='rb', right_on='reeds_ba_list')
    merged_gdf['country']='USA'
    merged_gdf=merged_gdf[['rb','state','country','geometry']]

    # Country - Group by 'country' and dissolve to merge geometries
    gdf_country = merged_gdf.dissolve(by='country').reset_index()

    # State - Group by 'state' and dissolve to merge geometries
    gdf_state = merged_gdf.dissolve(by='state').reset_index()

    # Subregion (rb) - No need to group since each 'rb' is already unique
    gdf_subregion = merged_gdf[['rb', 'geometry']].drop_duplicates()


    # Convert each GeoDataFrame to GeoJSON for use in Plotly
    geojson_country = json.loads(gdf_country.to_json())
    geojson_state = json.loads(gdf_state.to_json())
    geojson_subregion = json.loads(gdf_subregion.to_json())
    import json
    current_directory= os.path.join(current_directory, 'web_page_data')
    # Define file paths for saving
    geojson_country_path = os.path.join(current_directory, 'geojson_country.json')
    geojson_state_path = os.path.join(current_directory, 'geojson_state.json')
    geojson_subregion_path = os.path.join(current_directory, 'geojson_subregion.json')
    # Define file paths for saving
    gdf_country_path = os.path.join(current_directory, 'gdf_country.gpkg')
    gdf_state_path = os.path.join(current_directory, 'gdf_state.gpkg')
    gdf_subregion_path = os.path.join(current_directory, 'gdf_subregion.gpkg')

    # Save the GeoDataFrames
    gdf_country.to_file(gdf_country_path, driver='GPKG')
    gdf_state.to_file(gdf_state_path, driver='GPKG')
    gdf_subregion.to_file(gdf_subregion_path, driver='GPKG')

    # Function to save GeoJSON to a file
    def save_geojson(data, file_path):
        with open(file_path, 'w') as f:
            json.dump(data, f)

    # Save the GeoJSON data
    save_geojson(geojson_country, geojson_country_path)
    save_geojson(geojson_state, geojson_state_path)
    save_geojson(geojson_subregion, geojson_subregion_path)
    # Function to read GeoJSON from a file
    def read_geojson(file_path):
        with open(file_path) as f:
            return json.load(f)

    # Read the GeoJSON data
    geojson_country_read = read_geojson(geojson_country_path)
    geojson_state_read = read_geojson(geojson_state_path)
    geojson_subregion_read = read_geojson(geojson_subregion_path)
    print(geojson_country == geojson_country_read)
    print(geojson_state == geojson_state_read)
    print(geojson_subregion == geojson_subregion_read)
    # Read the GeoDataFrames
    gdf_country_read = gpd.read_file(gdf_country_path)
    gdf_state_read = gpd.read_file(gdf_state_path)
    gdf_subregion_read = gpd.read_file(gdf_subregion_path)
    print(gdf_country.shape == gdf_country_read.shape)
    print(gdf_state.shape == gdf_state_read.shape)
    print(gdf_subregion.shape == gdf_subregion_read.shape)

    # Sample a row and compare geometries (example using 'gdf_country')
    print(gdf_country.geometry.iloc[0].equals(gdf_country_read.geometry.iloc[0]))


## create lower the bound for future test
so we take the lowerest month in the training data and hottest month, teh 97.5 confidence inteval, the hottest month donate the upper bound, lowest month donate colde

In [13]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
for case in ['projection']:
    # Define the directory where the CSV files are located and other initial setups
    directory = f'/Volumes/T7/prediction/{case}'
    rb_codes = [f'p{i}' for i in range(1, 135)]
    years = list(range(2020, 2100))

    def string_to_set(string):
        elements = string.replace('{', '').replace('}', '').split(',')
        return set(e.strip().replace("'", "") for e in elements)

    def upper_95(x):
        return x.quantile(0.95)

    def lower_5(x):
        return x.quantile(0.05)

    def process_rb_files_hourly(rb_code, year):
        file_name = f"{rb_code}_{year}_mlp_output.csv"
        file_path = os.path.join(directory, str(year), file_name)
        if os.path.isfile(file_path):
            temp_df = pd.read_csv(file_path, usecols=['Time_UTC', 'Load'])
            temp_df['Time_UTC'] = pd.to_datetime(temp_df['Time_UTC'])
            temp_df[rb_code]=temp_df['Load']
            # temp_df['Hour'] = temp_df['Time_UTC'].dt.hour
            # temp_df['Year'] = temp_df['Time_UTC'].dt.year
            # temp_df['Weekend_or_Weekday'] = np.where(temp_df['Time_UTC'].dt.dayofweek < 5, 'Weekday', 'Weekend')
            # temp_df['Day'] = temp_df['Time_UTC'].dt.day
            # temp_df['Month'] = temp_df['Time_UTC'].dt.month
            temp_df.drop(columns=['Load'], inplace=True)
            return temp_df
        else:
            return pd.DataFrame()

    def aggregate_state_and_usa_daily(df, state_to_ba_mapping):
        for index, row in state_to_ba_mapping.iterrows():
            rb_list = string_to_set(row['reeds_ba_list'])
            state_column = row['state']
            df[state_column] = df[[col for col in df.columns if col in rb_list]].sum(axis=1)
        df['USA'] = df[[row['state'] for index, row in state_to_ba_mapping.iterrows()]].sum(axis=1)
        return df

    def process_yearly_data(year):
        state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')
        First = True

        for rb_code in rb_codes:
            temp_df = process_rb_files_hourly(rb_code, year)
            if First:
                processed_df=temp_df.copy()
                First=False
            else:
                processed_df = processed_df.merge(temp_df, on=['Time_UTC'], how='inner')
        yearly_data=processed_df 

        yearly_data = aggregate_state_and_usa_daily(yearly_data, state_to_ba_mapping)
        yearly_data['Hour'] = yearly_data['Time_UTC'].dt.hour
        yearly_data['Year'] = yearly_data['Time_UTC'].dt.year
        yearly_data['Weekend_or_Weekday'] = np.where(yearly_data['Time_UTC'].dt.dayofweek < 5, 'Weekday', 'Weekend')
        yearly_data.drop(columns=['Time_UTC'], inplace=True)
        columns_to_aggregate = [col for col in yearly_data.columns if col not in ['Hour', 'Year', 'Weekend_or_Weekday']]
        aggregations = {col: ['mean', upper_95, lower_5,'max'] for col in columns_to_aggregate}
            
        # Group by 'Hour', 'Year', 'Weekend_or_Weekday', then aggregate
        grouped_yearly_data = yearly_data.groupby(['Hour', 'Year', 'Weekend_or_Weekday']).agg(aggregations)
        # Flatten the MultiIndex columns if you have multiple columns being aggregated
        grouped_yearly_data.columns = ['_'.join(col).strip() for col in grouped_yearly_data.columns.values]

        # Reset index if you want 'Hour', 'Year', 'Weekend_or_Weekday' back as regular columns
        grouped_yearly_data.reset_index(inplace=True)
        grouped_yearly_data.rename(columns=lambda x: x.replace('mean', 'mean').replace('upper_95', 'upper').replace('lower_5', 'lower'), inplace=True)

        return grouped_yearly_data

    final_dfs = []
    for year in years:
        yearly_data = process_yearly_data(year)
        if not yearly_data.empty:
            final_dfs.append(yearly_data)

    final_df = pd.concat(final_dfs, ignore_index=True)
    final_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/mock_{case}_yearly_aggregated.csv')
    final_df

/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_54211/602522168.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_yearly_data.reset_index(inplace=True)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_54211/602522168.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_yearly_data.reset_index(inplace=True)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_54211/602522168.py:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert

In [14]:
for case in ['projection']:

    # Define the directory where the CSV files are located
    directory = f'/Volumes/T7/prediction/{case}'

    # Create a list of all 'rb' codes and years
    rb_codes = [f'p{i}' for i in range(1, 135)]
    years = list(range(2020, 2100))
    months = list(range(1, 13))


    def process_rb_files_daily(rb_code, years):
        rb_df_list = []
        for year in years:
            folder_path = os.path.join(directory, str(year))
            file_name = f"{rb_code}_{year}_mlp_output.csv"
            file_path = os.path.join(folder_path, file_name)
            if os.path.isfile(file_path):
                temp_df = pd.read_csv(file_path)[['Time_UTC', 'Load']]
                temp_df['Time_UTC'] = pd.to_datetime(temp_df['Time_UTC'])
                # Additional grouping by day
                temp_df['Day'] = temp_df['Time_UTC'].dt.day
                temp_df['Year'] = temp_df['Time_UTC'].dt.year
                temp_df['Month'] = temp_df['Time_UTC'].dt.month
                aggregated_df = temp_df.groupby(['Year', 'Month', 'Day'])['Load'].sum().reset_index()
                rb_df_list.append(aggregated_df)
        concatenated_df = pd.concat(rb_df_list, ignore_index=True)
        return concatenated_df
    def aggregate_state_and_usa_daily(final_rb_df):
        # Read the state to rb mapping
        state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')
        
        # For each state, sum the relevant 'rb' daily sums
        for index, row in state_to_ba_mapping.iterrows():
            rb_list = string_to_set(row['reeds_ba_list'])  # Convert string to list/set of rb codes
            state_column = row['state']
            # Ensure only existing columns are summed
            rb_columns = [rb for rb in rb_list if rb in final_rb_df.columns]
            # Summing for each state
            final_rb_df[state_column] = final_rb_df[rb_columns].sum(axis=1)
        
        # Summing all states to get USA total
        state_columns = state_to_ba_mapping['state'].tolist()
        final_rb_df['USA'] = final_rb_df[state_columns].sum(axis=1)
        
        return final_rb_df

    def upper_95(x):
        return x.quantile(0.95)

    def lower_5(x):
        return x.quantile(0.05)
    # Define a function to reorganize each column into an array grouped by Year
    def reorganize_into_array(series):
        # Pre-fill the array with NaNs to handle missing weekdays
        array = [np.nan] * 7  # One entry for each day of the week
        for idx, value in series.items():
            # idx is a tuple (Year, weekday), value is the column value
            # Subtract 1 from idx[1] if weekday starts from 1 in your dataset
            array[idx[1]] = value  # Or idx[1] - 1 if weekday is 1-based
        return array

    final_dfs = []
    for year in years:
        # Process data for the year (pseudo-code placeholders for actual processing)
        First=True
        for rb_code in rb_codes:
            temp_df = process_rb_files_daily(rb_code, [year])
            if First:
                processed_df=temp_df.copy()
                First=False
                processed_df[rb_code]=processed_df['Load']
                processed_df=processed_df.drop(['Load'], axis=1)
            else:
                # Assuming processed_df and temp_df are defined and have the appropriate columns
                processed_df = processed_df.merge(temp_df, on=['Year', 'Month', 'Day'], how='inner')
                processed_df[rb_code]=processed_df['Load']
                processed_df=processed_df.drop(['Load'], axis=1)
        processed_df = aggregate_state_and_usa_daily(processed_df)
        processed_df['Date'] = pd.to_datetime(processed_df[['Year', 'Month', 'Day']])

        # Create the 'weekday' column, note that dt.dayofweek returns Monday=0 through Sunday=6, so we add 1
        processed_df['weekday'] = processed_df['Date'].dt.dayofweek 
        processed_df=processed_df.drop(['Month','Day','Date'], axis=1)
        # Group by 'weekday' and 'Year', then apply the aggregations
        aggregations = {col: ['mean','max', upper_95, lower_5] for col in processed_df.columns if col not in ['Year', 'weekday']}
        grouped_df = processed_df.groupby(['Year', 'weekday']).agg(aggregations)
        # Now, flatten the MultiIndex in columns created by aggregation
        grouped_df.columns = ['{}_{}'.format(col[0], col[1]) for col in grouped_df.columns]

        # To further match your requirement, rename the aggregation methods in column names
        grouped_df.rename(columns=lambda x: x.replace('mean', 'mean').replace('upper_95', 'upper').replace('lower_5', 'lower'), inplace=True)

        # Reset index to turn 'Year' and 'weekday' back into columns if needed
        grouped_df= grouped_df.reset_index()
        # First, let's ensure 'weekday' is in the correct data type if not already
        grouped_df['weekday'] = grouped_df['weekday'].astype(int)
        final_dfs.append(grouped_df)
        

    final_df = pd.concat(final_dfs, axis=0, ignore_index=True)
    final_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/mock_{case}_weekly.csv')

In [15]:

import pandas as pd
import os
from datetime import datetime
import numpy as np
# Placeholder function to read 'state_to_ba_mapping.csv' and perform aggregation
for case in ['projection']:
    def string_to_set(string):
        # Remove curly braces and split the string
        elements = string.replace('{', '').replace('}', '').split(',')
        # Remove any extra whitespace and single quotes from each element
        elements = [e.strip().replace("'", "") for e in elements]
        return set(elements)
    # Define the directory where the CSV files are located
    directory = f'/Volumes/T7/prediction/{case}'

    # Create a list of all 'rb' codes and years
    rb_codes = [f'p{i}' for i in range(1, 135)]
    years = list(range(2020, 2100))
    months = list(range(1, 13))


    def process_rb_files_daily(rb_code, years):
        rb_df_list = []
        for year in years:
            folder_path = os.path.join(directory, str(year))
            file_name = f"{rb_code}_{year}_mlp_output.csv"
            file_path = os.path.join(folder_path, file_name)
            if os.path.isfile(file_path):
                temp_df = pd.read_csv(file_path)[['Time_UTC', 'Load']]
                temp_df['Time_UTC'] = pd.to_datetime(temp_df['Time_UTC'])
                rb_df_list.append(temp_df)
        concatenated_df = pd.concat(rb_df_list, ignore_index=True)
        return concatenated_df
    def aggregate_state_and_usa_daily(final_rb_df):
        # Read the state to rb mapping
        state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')
        
        # For each state, sum the relevant 'rb' daily sums
        for index, row in state_to_ba_mapping.iterrows():
            rb_list = string_to_set(row['reeds_ba_list'])  # Convert string to list/set of rb codes
            state_column = row['state']
            # Ensure only existing columns are summed
            rb_columns = [rb for rb in rb_list if rb in final_rb_df.columns]
            # Summing for each state
            final_rb_df[state_column] = final_rb_df[rb_columns].sum(axis=1)
        
        # Summing all states to get USA total
        state_columns = state_to_ba_mapping['state'].tolist()
        final_rb_df['USA'] = final_rb_df[state_columns].sum(axis=1)
        
        return final_rb_df



    final_dfs = []
    for year in years:
        # Process data for the year (pseudo-code placeholders for actual processing)
        First=True
        for rb_code in rb_codes:
            temp_df = process_rb_files_daily(rb_code, [year])
            if First:
                processed_df=temp_df.copy()
                First=False
                processed_df[rb_code]=processed_df['Load']
                processed_df=processed_df.drop(['Load'], axis=1)
            else:
                # Assuming processed_df and temp_df are defined and have the appropriate columns
                processed_df = processed_df.merge(temp_df, on=['Time_UTC'], how='inner')
                processed_df[rb_code]=processed_df['Load']
                processed_df=processed_df.drop(['Load'], axis=1)
        processed_df = aggregate_state_and_usa_daily(processed_df)
        processed_df['Time_UTC'] = pd.to_datetime(processed_df['Time_UTC'])
        processed_df['Year'] = processed_df['Time_UTC'].dt.year
        processed_df['Month'] = processed_df['Time_UTC'].dt.month


        processed_df=processed_df.drop(['Time_UTC'], axis=1)
        # Group by 'weekday' and 'Year', then apply the aggregations
        # Define aggregations to apply 'max' to all columns except 'Year', 'Month', and 'weekday'
        aggregations = {col: 'max' for col in processed_df.columns if col not in ['Year', 'Month']}

        # Group by 'Year' and 'Month', then aggregate
        grouped_df = processed_df.groupby(['Year', 'Month']).agg(aggregations)



        # Reset index to turn 'Year' and 'Month' back into columns if needed
        grouped_df.reset_index(inplace=True)
        final_dfs.append(grouped_df)


        

    final_df = pd.concat(final_dfs, axis=0, ignore_index=True)
    final_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/max_{case}_monthlly.csv')
    final_df

In [10]:

import pandas as pd
import os
from datetime import datetime
cases = ['projection']
for case in cases:
    # Define the directory where the CSV files are located
    directory = f'/Volumes/T7/prediction/{case}'

    # Create a list of all 'rb' codes and years
    rb_codes = [f'p{i}' for i in range(1, 135)]



    # Function to read and process files for a given rb code
    def process_rb_files(rb_code):
        rb_df_list = []
        # Loop through all years for the given rb_code
        for year in years:
            # Construct file name
            folder_path = os.path.join(directory, str(year))
            file_name = f"{rb_code}_{year}_mlp_output.csv"
            file_path = os.path.join(folder_path, file_name)
            # Check if file exists to avoid FileNotFoundError
            if os.path.isfile(file_path):
                # Read the CSV file
                temp_df = pd.read_csv(file_path)
                temp_df=temp_df[['Time_UTC','Load']]
                # Convert 'Time_UTC' to datetime and extract 'Year' and 'Month'
                temp_df['Time_UTC'] = pd.to_datetime(temp_df['Time_UTC'])
                temp_df['Year'] = temp_df['Time_UTC'].dt.year
                temp_df['Month'] = temp_df['Time_UTC'].dt.month
                # Aggregate Load by 'Year' and 'Month'
                aggregated_df = temp_df.groupby(['Year', 'Month'])['Load'].sum().reset_index()
                # Add the aggregated data to the list
                rb_df_list.append(aggregated_df)
        
        # Vertically concatenate all yearly data for the rb code
        concatenated_df = pd.concat(rb_df_list, ignore_index=True)
        return concatenated_df

    years = list(range(2020, 2100))
    months = list(range(1, 13))

    # Create a DataFrame with all combinations of 'Year' and 'Month'
    year_month_df = pd.DataFrame([(y, m) for y in years for m in months], columns=['Year', 'Month'])

    # Placeholder to simulate merging data for multiple rb codes
    final_df = year_month_df.copy()
    for rb_code in rb_codes: 
        rb_df = process_rb_files(rb_code)
        final_df = final_df.merge(rb_df, on=['Year', 'Month'], how='left').rename(columns={'Load': rb_code})
    # Placeholder function to read 'state_to_ba_mapping.csv' and perform aggregation
    def string_to_set(string):
        # Remove curly braces and split the string
        elements = string.replace('{', '').replace('}', '').split(',')
        # Remove any extra whitespace and single quotes from each element
        elements = [e.strip().replace("'", "") for e in elements]
        return set(elements)
    def aggregate_state_data(final_df):
        # Placeholder to simulate reading the 'state_to_ba_mapping.csv' file
        # In practice, this should read the actual file
        state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')

        # Add columns for each state by summing the relevant rb columns based on the mapping
        for index, row in state_to_ba_mapping.iterrows():
            rb_list = string_to_set(row['reeds_ba_list'])
            state_column = row['state']
            # Sum the columns specified in reeds_ba_list for each state
            final_df[state_column] = final_df[list(rb_list)].sum(axis=1)

        # Calculate the total for the USA by summing all state columns
        state_columns = state_to_ba_mapping['state'].values
        final_df['USA'] = final_df[state_columns].sum(axis=1)

        return final_df

    # Apply the placeholder aggregation function to the simulated final_df
    final_aggregated_df = aggregate_state_data(final_df)

    def aggregate_state_data(final_df):
        # Placeholder to simulate reading the 'state_to_ba_mapping.csv' file
        # In practice, this should read the actual file
        state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')

        # Add columns for each state by summing the relevant rb columns based on the mapping
        for index, row in state_to_ba_mapping.iterrows():
            rb_list = string_to_set(row['reeds_ba_list'])
            state_column = row['state']
            # Sum the columns specified in reeds_ba_list for each state
            final_df[state_column] = final_df[list(rb_list)].sum(axis=1)

        # Calculate the total for the USA by summing all state columns
        state_columns = state_to_ba_mapping['state'].values
        final_df['USA'] = final_df[state_columns].sum(axis=1)

        return final_df

    # Apply the placeholder aggregation function to the simulated final_df
    final_aggregated_df = aggregate_state_data(final_df)
    final_aggregated_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/mock_{case}csv')

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as st
import os

# Define the directory where your CSV files are located
directory = '/Users/ansonkong/Downloads/Data for nyu work/output/demand_weather_combine_2007'
ci_output_directory = os.path.join('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources', 'CI_results')
os.makedirs(ci_output_directory, exist_ok=True)  # Ensure the output directory exists

def calculate_quantile_ci(data, confidence=0.95):
    """
    Calculate the confidence interval using quantiles for a given dataset.
    """
    # Calculate the lower and upper percentile bounds for the confidence interval
    lower_percentile = 100 * (1 - confidence) / 2
    upper_percentile = 100 - lower_percentile
    
    # Use np.percentile to find the lower and upper quantiles
    ci_lower = np.percentile(data, lower_percentile)
    ci_upper = np.percentile(data, upper_percentile)
    
    return ci_lower, ci_upper

# List all files in the directory
files = [file for file in os.listdir(directory) if file.endswith('.csv')]

for file in files:
    filepath = os.path.join(directory, file)
    df = pd.read_csv(filepath)
    
    # Extract 'rb' from filename
    rb_code = file.split('_')[0]
    
    # Convert Time_UTC to datetime and extract Year, Month, and Day
    df['Time_UTC'] = pd.to_datetime(df['Time_UTC'])
    df['Month'] = df['Time_UTC'].dt.month
    
    # Initialize lists to store CI results
    ci_results_max = []
    ci_results_min = []
    
    for month in range(1, 13):
        month_data = df[df['Month'] == month]
        daily_max = month_data.groupby(month_data['Time_UTC'].dt.date)['T2'].max()
        daily_min = month_data.groupby(month_data['Time_UTC'].dt.date)['T2'].min()
        
        ci_max_lower, ci_max_upper = calculate_quantile_ci(daily_max)
        ci_min_lower, ci_min_upper = calculate_quantile_ci(daily_min)
        
        ci_results_max.append([rb_code, ci_max_lower, ci_max_upper])
        ci_results_min.append([rb_code, ci_min_lower, ci_min_upper])
    
    # Convert CI results to DataFrames
    ci_df_max = pd.DataFrame(ci_results_max, columns=['rb',  'CI_max_lower', 'CI_max_upper'])
    ci_df_min = pd.DataFrame(ci_results_min, columns=['rb',  'CI_min_lower', 'CI_min_upper'])
    # For ci_df_max, keep only the maximum upper CI for each rb
    ci_df_max_reduced = ci_df_max.groupby('rb')['CI_max_upper'].max().reset_index()

    # For ci_df_min, keep only the minimum lower CI for each rb
    ci_df_min_reduced = ci_df_min.groupby('rb')['CI_min_lower'].min().reset_index()

    # Reindex the DataFrames
    # ci_df_max_reduced.reset_index(drop=True, inplace=True)
    # ci_df_min_reduced.reset_index(drop=True, inplace=True)
    
    # Save CI DataFrames to CSV files
    ci_df_max_reduced.to_csv(os.path.join(ci_output_directory, f'{rb_code}_CI_max.csv'), index=False)
    ci_df_min_reduced.to_csv(os.path.join(ci_output_directory, f'{rb_code}_CI_min.csv'), index=False)
# Define the directory where your CSV files are located
directory = '/Users/ansonkong/Downloads/Data for nyu work/output/demand_weather_combine_2007'
ci_output_directory = os.path.join('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources', 'CI_results')
os.makedirs(ci_output_directory, exist_ok=True)  # Ensure the output directory exists

def calculate_quantile_ci(data, confidence=0.95):
    """
    Calculate the confidence interval using quantiles for a given dataset.
    """
    # Calculate the lower and upper percentile bounds for the confidence interval
    lower_percentile = 100 * (1 - confidence) / 2
    upper_percentile = 100 - lower_percentile
    
    # Use np.percentile to find the lower and upper quantiles
    ci_lower = np.percentile(data, lower_percentile)
    ci_upper = np.percentile(data, upper_percentile)
    
    return ci_lower, ci_upper

# List all files in the directory
files = [file for file in os.listdir(directory) if file.endswith('.csv')]
states=['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
           'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
          'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
          'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
          'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming','usa']
rb_codes = [f'p{i}' for i in range(1, 135)]
for state in states:
    state_df=pd.DataFrame()
    for year in range(2007,2013):
        file=f'{state}_averaged_weather_{year}.csv'

        filepath = os.path.join('/Users/ansonkong/Downloads/Data for nyu work/averaged_historical_weather', file)
        df = pd.read_csv(filepath)
        # Concatenate this DataFrame with the accumulating state_df
        state_df = pd.concat([state_df, df], ignore_index=True)
    
    
    # Convert Time_UTC to datetime and extract Year, Month, and Day
    state_df['Time_UTC'] = pd.to_datetime(state_df['Time_UTC'])
    state_df['Month'] = state_df['Time_UTC'].dt.month
    
    # Initialize lists to store CI results
    ci_results_max = []
    ci_results_min = []
    
    for month in range(1, 13):
        month_data = state_df[state_df['Month'] == month]
        daily_max = month_data.groupby(month_data['Time_UTC'].dt.date)['T2'].max()
        daily_min = month_data.groupby(month_data['Time_UTC'].dt.date)['T2'].min()
        
        ci_max_lower, ci_max_upper = calculate_quantile_ci(daily_max)
        ci_min_lower, ci_min_upper = calculate_quantile_ci(daily_min)
        
        ci_results_max.append([ state, ci_max_lower, ci_max_upper])
        ci_results_min.append([ state, ci_min_lower, ci_min_upper])
    
    # Convert CI results to DataFrames
    ci_df_max = pd.DataFrame(ci_results_max, columns=['State',  'CI_max_lower', 'CI_max_upper'])
    ci_df_min = pd.DataFrame(ci_results_min, columns=['State',  'CI_min_lower', 'CI_min_upper'])
    # For ci_df_max, keep only the maximum upper CI for each rb
    ci_df_max_reduced = ci_df_max.groupby('State')['CI_max_upper'].max().reset_index()

    # For ci_df_min, keep only the minimum lower CI for each rb
    ci_df_min_reduced = ci_df_min.groupby('State')['CI_min_lower'].min().reset_index()


    # Save CI DataFrames to CSV files
    ci_df_max_reduced.to_csv(os.path.join(ci_output_directory, f'{state}_CI_max.csv'), index=False)
    ci_df_min_reduced.to_csv(os.path.join(ci_output_directory, f'{state}_CI_min.csv'), index=False)

In [95]:
# import pandas as pd
# import numpy as np
# import os

# # Directories initialization
# directory = '/Users/ansonkong/Downloads/Data for nyu work/averaged_rcp85hotter_weather'
# ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
# output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'
# os.makedirs(output_directory, exist_ok=True)

# states=['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
#            'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
#           'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
#           'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
#           'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming','usa']
# years = range(2020, 2100)

# extreme_outliers_max_list = []
# extreme_outliers_min_list = []

# for state in states:
#     extreme_outliers_max_list = []
#     extreme_outliers_min_list = []
#     ci_max_path = os.path.join(ci_directory, f'{state}_CI_max.csv')
#     ci_min_path = os.path.join(ci_directory, f'{state}_CI_min.csv')
    
#     if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
#         ci_max_df = pd.read_csv(ci_max_path)
#         ci_min_df = pd.read_csv(ci_min_path)
        
#         for year in years:
#             weather_file_path = os.path.join(directory, f'{state}_averaged_weather_{year}.csv')
#             if os.path.exists(weather_file_path):
#                 df = pd.read_csv(weather_file_path)
#                 df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
#                 df['Month'] = pd.to_datetime(df['Time_UTC']).dt.month
                    
#                 # Only proceed if the month matches
#                 ci_max_month = ci_max_df[ (ci_max_df['State'] == state)]
#                 ci_min_month = ci_min_df[ (ci_min_df['State'] == state)]
                    
#                 # Find the dates with the highest and lowest T2 for each day
#                 grouped = df.groupby('Date')['T2'].agg(['max', 'min'])
#                 dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist()
#                 # print(ci_max_df)
#                 # print(grouped)
#                 # print(grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist())
#                 for date in dates_above_max_ci:
#                     extreme_outliers_max_list.append({'rb': rb_code, 'Year': year, 'Date': date})

#                 # Find all dates where min T2 is below the CI_min_lower
#                 dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]].index.tolist()
#                 for date in dates_below_min_ci:
#                     extreme_outliers_min_list.append({'rb': rb_code, 'Year': year, 'Date': date})

#     # Convert lists to DataFrames
#     extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
#     extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)

#     # Save the results
#     extreme_outliers_max.to_csv(os.path.join(output_directory, f'{state}_extreme_outliers_max.csv'), index=False)
#     extreme_outliers_min.to_csv(os.path.join(output_directory, f'{state}_extreme_outliers_min.csv'), index=False)

In [4]:
import pandas as pd
import numpy as np
import os
cases = ['projection']#'rcp85hotter', 'rcp45hotter', 'rcp85cooler', 'rcp45cooler',
for case in cases:#, 'rcp45hotter', 'rcp45cooler']
    # Directories initialization
    directory = f'/Users/ansonkong/Downloads/Data for nyu work/output/future_{case}_weather'
    ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
    output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'
    os.makedirs(output_directory, exist_ok=True)

    rb_codes = [f'p{i}' for i in range(1, 135)]
    years = range(2020, 2100)

  
    for rb_code in rb_codes:
        extreme_outliers_max_list = []
        extreme_outliers_min_list = []
        hdd_list = []
        cdd_list = []
        ci_max_path = os.path.join(ci_directory, f'{rb_code}_CI_max.csv')
        ci_min_path = os.path.join(ci_directory, f'{rb_code}_CI_min.csv')
        
        if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
            ci_max_df = pd.read_csv(ci_max_path)
            ci_min_df = pd.read_csv(ci_min_path)
            
            for year in years:
                weather_file_path = os.path.join(directory, f'{rb_code}_WRF_Hourly_Mean_Meteorology_{year}.csv')
                if os.path.exists(weather_file_path):
                    df = pd.read_csv(weather_file_path)
                    df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                    df['Month'] = pd.to_datetime(df['Time_UTC']).dt.month
                        
                    # Only proceed if the month matches
                    ci_max_month = ci_max_df[ (ci_max_df['rb'] == rb_code)]
                    ci_min_month = ci_min_df[ (ci_min_df['rb'] == rb_code)]
                        
                    # Find the dates with the highest and lowest T2 for each day
                    grouped = df.groupby('Date')['T2'].agg(['max', 'min'])
                    dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist()
                    # print(ci_max_df)
                    # print(grouped)
                    # print(grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist())
                    for date in dates_above_max_ci:
                        extreme_outliers_max_list.append({'rb': rb_code, 'Year': year, 'Date': date})

                    # Find all dates where min T2 is below the CI_min_lower
                    dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]].index.tolist()
                    for date in dates_below_min_ci:
                        extreme_outliers_min_list.append({'rb': rb_code, 'Year': year, 'Date': date})

        # Convert lists to DataFrames
        extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
        extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)

        # Save the results
        extreme_outliers_max.to_csv(os.path.join(output_directory, f'{rb_code}_extreme_outliers_max_{case}.csv'), index=False)
        extreme_outliers_min.to_csv(os.path.join(output_directory, f'{rb_code}_extreme_outliers_min_{case}.csv'), index=False)
        for year in years:
            weather_file_path = os.path.join(directory, f'{rb_code}_WRF_Hourly_Mean_Meteorology_{year}.csv')
            hdd=0
            cdd=0
            if os.path.exists(weather_file_path):
                df = pd.read_csv(weather_file_path)
                df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                        
   
                # Find the dates with the highest and lowest T2 for each day
                grouped = df.groupby('Date')['T2'].agg(['mean'])
                dates_above_hdd = grouped[grouped['mean'] > 291.45]
                for index, row in dates_above_hdd.iterrows():
                    hdd+=row['mean']-291.45
                if not hdd:
                    hdd_list.append({'rb': rb_code, 'Year': year, 'hdd': 0.0})
                else:
                    hdd_list.append({'rb': rb_code, 'Year': year, 'hdd': hdd})

                # Find all dates where min T2 is below the CI_min_lower
                dates_below_cdd = grouped[grouped['mean'] < 291.45]
                for index, row in dates_below_cdd.iterrows():
                    cdd+=291.45-row['mean']
                cdd_list.append({'rb': rb_code, 'Year': year, 'cdd': cdd})
         # Convert lists to DataFrames
        hdd_list = pd.DataFrame(hdd_list)
        cdd_list = pd.DataFrame(cdd_list)

        # Save the results
        hdd_list.to_csv(os.path.join(output_directory, f'{rb_code}_hdd_{case}.csv'), index=False)
        cdd_list.to_csv(os.path.join(output_directory, f'{rb_code}_cdd_{case}.csv'), index=False)





    # Directories initialization
    directory = f'/Users/ansonkong/Downloads/Data for nyu work/averaged_{case}_weather'
    ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
    output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'
    os.makedirs(output_directory, exist_ok=True)

    states=['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
            'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
            'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
            'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
            'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming','usa']
    years = range(2020, 2100)

    extreme_outliers_max_list = []
    extreme_outliers_min_list = []

    for state in states:
        extreme_outliers_max_list = []
        extreme_outliers_min_list = []
        hdd_list = []
        cdd_list = []
        ci_max_path = os.path.join(ci_directory, f'{state}_CI_max.csv')
        ci_min_path = os.path.join(ci_directory, f'{state}_CI_min.csv')
        
        if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
            ci_max_df = pd.read_csv(ci_max_path)
            ci_min_df = pd.read_csv(ci_min_path)
            
            for year in years:
                weather_file_path = os.path.join(directory, f'{state}_averaged_weather_{year}.csv')
                if os.path.exists(weather_file_path):
                    df = pd.read_csv(weather_file_path)
                    df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                    df['Month'] = pd.to_datetime(df['Time_UTC']).dt.month
                        
                    # Only proceed if the month matches
                    ci_max_month = ci_max_df[ (ci_max_df['State'] == state)]
                    ci_min_month = ci_min_df[ (ci_min_df['State'] == state)]
                        
                    # Find the dates with the highest and lowest T2 for each day
                    grouped = df.groupby('Date')['T2'].agg(['max', 'min'])
                    dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist()
                    # print(ci_max_df)
                    # print(grouped)
                    # print(grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist())
                    for date in dates_above_max_ci:
                        extreme_outliers_max_list.append({'state': state, 'Year': year, 'Date': date})

                    # Find all dates where min T2 is below the CI_min_lower
                    dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]].index.tolist()
                    for date in dates_below_min_ci:
                        extreme_outliers_min_list.append({'state': state, 'Year': year, 'Date': date})
        for year in years:
            weather_file_path = os.path.join(directory, f'{state}_averaged_weather_{year}.csv')
            hdd=0
            cdd=0

            if os.path.exists(weather_file_path):
                df = pd.read_csv(weather_file_path)
                df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
            
                # Find the dates with the highest and lowest T2 for each day
                grouped = df.groupby('Date')['T2'].agg(['mean'])
                dates_above_hdd = grouped[grouped['mean'] > 291.45]
                for index, row in dates_above_hdd.iterrows():
                    hdd+=row['mean']-291.45
                hdd_list.append({'state': state, 'Year': year, 'hdd': hdd})

                # Find all dates where min T2 is below the CI_min_lower
                dates_below_cdd = grouped[grouped['mean'] < 291.45]
                for index, row in dates_below_cdd.iterrows():
                    cdd+=291.45-row['mean']
                cdd_list.append({'state': state, 'Year': year, 'cdd': cdd})

        # Convert lists to DataFrames
        extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
        extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)

        # Save the results
        extreme_outliers_max.to_csv(os.path.join(output_directory, f'{state}_extreme_outliers_max_{case}.csv'), index=False)
        extreme_outliers_min.to_csv(os.path.join(output_directory, f'{state}_extreme_outliers_min_{case}.csv'), index=False)


        # Convert lists to DataFrames
        hdd_list = pd.DataFrame(hdd_list)
        cdd_list = pd.DataFrame(cdd_list)

        # Save the results
        hdd_list.to_csv(os.path.join(output_directory, f'{state}_hdd_{case}.csv'), index=False)
        cdd_list.to_csv(os.path.join(output_directory, f'{state}_cdd_{case}.csv'), index=False)



    # Specify the folder
    folder = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'

    # Initialize an empty DataFrame to store all results
    all_results_df = pd.DataFrame()

    # List all "max" files in the folder
    max_files = [f for f in os.listdir(folder) if f.endswith(f'_extreme_outliers_max_{case}.csv')]

    for file in max_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_extreme_outliers_max_{case}.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        # Group by 'Year' and count the number of observations for each year
        year_counts = df.groupby('Year').size().reset_index(name='number_of_days')
        
        # Add the region to the DataFrame
        year_counts['region'] = region
        
        # Append the result to the all_results_df DataFrame
        all_results_df = pd.concat([all_results_df, year_counts], ignore_index=True)

    # Reorder the DataFrame columns if needed
    all_results_df = all_results_df[['region', 'Year', 'number_of_days']]

    # You now have a DataFrame containing the region, Year, and number_of_days for all "max" files
    # Optionally, save this DataFrame to a new CSV file
    all_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_max_outliers_summary_{case}.csv', index=False)


    # Initialize an empty DataFrame to store all results for minimum outliers
    all_min_results_df = pd.DataFrame()

    # List all "min" files in the folder
    min_files = [f for f in os.listdir(folder) if f.endswith(f'_extreme_outliers_min_{case}.csv')]

    for file in min_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_extreme_outliers_min_{case}.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        
        # Group by 'Year' and count the number of observations for each year
        year_counts = df.groupby('Year').size().reset_index(name='number_of_days')
        
        # Add the region to the DataFrame
        year_counts['region'] = region
        
        # Append the result to the all_min_results_df DataFrame
        all_min_results_df = pd.concat([all_min_results_df, year_counts], ignore_index=True)

    # Reorder the DataFrame columns if needed
    all_min_results_df = all_min_results_df[['region', 'Year', 'number_of_days']]

    # You now have a DataFrame containing the region, Year, and number_of_days for all "min" files
    # Optionally, save this DataFrame to a new CSV file
    all_min_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_min_outliers_summary_{case}.csv', index=False)

    # List all "hdd" files in the folder
    hdd_files = [f for f in os.listdir(folder) if f.endswith(f'_hdd_{case}.csv')]

    for file in hdd_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_hdd_{case}.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        
        # Add the region to the DataFrame
        df['region'] = region

        df=df[['Year','region','hdd']]
        
        # Append the result to the all_results_df DataFrame
        all_results_df = pd.concat([all_results_df, df], ignore_index=True)

    # Reorder the DataFrame columns if needed
    all_results_df = all_results_df[['region', 'Year', 'hdd']]

    # You now have a DataFrame containing the region, Year, and number_of_days for all "max" files
    # Optionally, save this DataFrame to a new CSV file
    all_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_hdd_{case}.csv', index=False)


    # Initialize an empty DataFrame to store all results for minimum outliers
    all_min_results_df = pd.DataFrame()

    # List all "min" files in the folder
    cdd_files = [f for f in os.listdir(folder) if f.endswith(f'_cdd_{case}.csv')]

    for file in cdd_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_cdd_{case}.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        
        # Add the region to the DataFrame
        df['region'] = region

        df=df[['Year','region','cdd']]
        
        # Append the result to the all_min_results_df DataFrame
        all_min_results_df = pd.concat([all_min_results_df, df], ignore_index=True)

    # Reorder the DataFrame columns if needed
    all_min_results_df = all_min_results_df[['region', 'Year', 'cdd']]

    # You now have a DataFrame containing the region, Year, and number_of_days for all "min" files
    # Optionally, save this DataFrame to a new CSV file
    all_min_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_cdd_{case}.csv', index=False)

Skipping empty file: p9_extreme_outliers_max_projection.csv
Skipping empty file: p20_extreme_outliers_max_projection.csv
Skipping empty file: p67_extreme_outliers_max_projection.csv
Skipping empty file: Wyoming_extreme_outliers_max_projection.csv
Skipping empty file: Wisconsin_extreme_outliers_max_projection.csv
Skipping empty file: p50_extreme_outliers_max_projection.csv
Skipping empty file: p17_extreme_outliers_max_projection.csv
Skipping empty file: Illinois_extreme_outliers_max_projection.csv
Skipping empty file: Minnesota_extreme_outliers_max_projection.csv
Skipping empty file: p74_extreme_outliers_max_projection.csv
Skipping empty file: p33_extreme_outliers_max_projection.csv
Skipping empty file: p108_extreme_outliers_max_projection.csv
Skipping empty file: p43_extreme_outliers_max_projection.csv
Skipping empty file: p107_extreme_outliers_max_projection.csv
Skipping empty file: p85_extreme_outliers_max_projection.csv
Skipping empty file: p130_extreme_outliers_max_projection.csv
S

average demand on extreme day

In [5]:
import pandas as pd
import numpy as np
import os
def string_to_set(string):
    # Remove curly braces and split the string
    elements = string.replace('{', '').replace('}', '').split(',')
    # Remove any extra whitespace and single quotes from each element
    elements = [e.strip().replace("'", "") for e in elements]
    return set(elements)

cases = ['projection']
for case in cases:#, 'rcp45hotter', 'rcp45cooler']
    directory = f'/Users/ansonkong/Downloads/Data for nyu work/output/future_{case}_weather'
    ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
    output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier_demand'
    os.makedirs(output_directory, exist_ok=True)

    rb_codes = [f'p{i}' for i in range(1, 135)]
    years = range(2020, 2100)

    for rb_code in rb_codes:
        extreme_outliers_max_list = []
        extreme_outliers_min_list = []

        ci_max_path = os.path.join(ci_directory, f'{rb_code}_CI_max.csv')
        ci_min_path = os.path.join(ci_directory, f'{rb_code}_CI_min.csv')
        
        if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
            ci_max_df = pd.read_csv(ci_max_path)
            ci_min_df = pd.read_csv(ci_min_path)
            
            for year in years:
                weather_file_path = os.path.join(directory, f'{rb_code}_WRF_Hourly_Mean_Meteorology_{year}.csv')
                demand_file_path = f'/Volumes/T7/prediction/{case}/{year}/{rb_code}_{year}_mlp_output.csv'

                if os.path.exists(weather_file_path) and os.path.exists(demand_file_path):
                    df = pd.read_csv(weather_file_path)
                    demand_df = pd.read_csv(demand_file_path)
                    
                    df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                    demand_df['Date'] = pd.to_datetime(demand_df['Time_UTC']).dt.date
                    
                    grouped = df.groupby('Date')['T2'].agg(['max', 'min'])
                    grouped_demand = demand_df.groupby('Date')['Load'].agg(['sum'])  # Changed to sum
                    
                    # Merge to get Load sum values for dates above and below CI thresholds
                    grouped = grouped.merge(grouped_demand, left_index=True, right_index=True, how='left')
                    
                    ci_max_month = ci_max_df[(ci_max_df['rb'] == rb_code)]
                    ci_min_month = ci_min_df[(ci_min_df['rb'] == rb_code)]
                    
                    dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]]
                    dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]]
                    
                    for date, row in dates_above_max_ci.iterrows():
                        extreme_outliers_max_list.append({'rb': rb_code, 'Year': year, 'Date': date, 'Load_sum': row['sum']})
                    
                    for date, row in dates_below_min_ci.iterrows():
                        extreme_outliers_min_list.append({'rb': rb_code, 'Year': year, 'Date': date, 'Load_sum': row['sum']})
        
        # Convert lists to DataFrames
        extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
        extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)
        
        # Save the results
        extreme_outliers_max.to_csv(os.path.join(output_directory, f'{rb_code}_extreme_demand_outliers_max_{case}.csv'), index=False)
        extreme_outliers_min.to_csv(os.path.join(output_directory, f'{rb_code}_extreme_demand_outliers_min_{case}.csv'), index=False)


    # Directories initialization
    directory = f'/Users/ansonkong/Downloads/Data for nyu work/averaged_{case}_weather'
    ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
    output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier_demand'
    os.makedirs(output_directory, exist_ok=True)

    states=['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
            'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
            'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
            'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
            'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming','usa']
    years = range(2020, 2100)

    extreme_outliers_max_list = []
    extreme_outliers_min_list = []

    for state in states:
        extreme_outliers_max_list = []
        extreme_outliers_min_list = []
        ci_max_path = os.path.join(ci_directory, f'{state}_CI_max.csv')
        ci_min_path = os.path.join(ci_directory, f'{state}_CI_min.csv')
        
        if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
            ci_max_df = pd.read_csv(ci_max_path)
            ci_min_df = pd.read_csv(ci_min_path)

            
            for year in years:
                all_rb_dfs = []
                weather_file_path = os.path.join(directory, f'{state}_averaged_weather_{year}.csv')
                if os.path.exists(weather_file_path):
                    if state == 'usa':
                        # If state is USA, set rb_list to include all RB codes from p1 to p134
                        rb_list = [f'p{i}' for i in range(1, 135)]
                    else:
                        state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')
                        state_to_ba_mapping=state_to_ba_mapping[state_to_ba_mapping['state']==state]
                        
                        rb_list = string_to_set(state_to_ba_mapping.iloc[0]['reeds_ba_list'])
                    for rb_code in list(rb_list):
                        demand_file_path = f'/Volumes/T7/prediction/{case}/{year}/{rb_code}_{year}_mlp_output.csv'
                        demand_df = pd.read_csv(demand_file_path)
                    
                        demand_df['Date'] = pd.to_datetime(demand_df['Time_UTC']).dt.date
                        
                        grouped_demand = demand_df.groupby('Date')['Load'].agg(['sum']).rename(columns={'sum': rb_code})  # Rename to allow identification

                        all_rb_dfs.append(grouped_demand)  
                    if all_rb_dfs:
                        total_load_df = pd.concat(all_rb_dfs, axis=1)

                        # Sum across all numerical columns for each date to calculate 'Total_Load'
                        total_load_df['Total_Load'] = total_load_df.select_dtypes(include=[np.number]).sum(axis=1)

                        total_load_df.reset_index(inplace=True)  # Reset index to turn 'Date' back into a column
                        
                        

                    df = pd.read_csv(weather_file_path)
                    df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                    # Find the dates with the highest and lowest T2 for each day
                    grouped = df.groupby('Date')['T2'].agg(['max', 'min'])

                    total_load_df.set_index('Date', inplace=True)

                    # Merge to get Load sum values for dates above and below CI thresholds
                    grouped = grouped.merge(total_load_df, left_index=True, right_index=True, how='left')
                        
                    # Only proceed if the month matches
                    ci_max_month = ci_max_df[ (ci_max_df['State'] == state)]
                    ci_min_month = ci_min_df[ (ci_min_df['State'] == state)]
                        
                    
                    dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]]
                    dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]]

 
                    for date, row in dates_above_max_ci.iterrows():
                        extreme_outliers_max_list.append({'rb': rb_code, 'Year': year, 'Date': date, 'Load_sum': row['Total_Load']})
                    
                    for date, row in dates_below_min_ci.iterrows():
                        extreme_outliers_min_list.append({'rb': rb_code, 'Year': year, 'Date': date, 'Load_sum': row['Total_Load']})

        # Convert lists to DataFrames
        extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
        extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)

        # Save the results
        extreme_outliers_max.to_csv(os.path.join(output_directory, f'{state}_extreme_demand_outliers_max_{case}.csv'), index=False)
        extreme_outliers_min.to_csv(os.path.join(output_directory, f'{state}_extreme_demand_outliers_min_{case}.csv'), index=False)
    # Initialize an empty DataFrame to store all results
    all_results_df = pd.DataFrame()
    folder = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier_demand'

    # List all "max" files in the folder
    max_files = [f for f in os.listdir(folder) if f.endswith(f'_extreme_demand_outliers_max_{case}.csv')]

    for file in max_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_extreme_demand_outliers_max_{case}.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        # Assuming 'Total_Load' is a column in 'df'
        # Group by 'Year' and calculate the average 'Total_Load' for each year
        year_average_total_load = df.groupby('Year')['Load_sum'].mean().reset_index(name='average_total_load')

        # Add the region to the DataFrame
        year_average_total_load['region'] = region

        # Append the result to the all_results_df DataFrame
        all_results_df = pd.concat([all_results_df, year_average_total_load], ignore_index=True)


    # Reorder the DataFrame columns if needed
    all_results_df = all_results_df[['region', 'Year', 'average_total_load']]

    # You now have a DataFrame containing the region, Year, and number_of_days for all "max" files
    # Optionally, save this DataFrame to a new CSV file
    all_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_max_outliers_demand_summary_{case}.csv', index=False)
    all_results_df= pd.DataFrame()


    # List all "min" files in the folder
    min_files = [f for f in os.listdir(folder) if f.endswith(f'_extreme_demand_outliers_min_{case}.csv')]

    for file in min_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_extreme_demand_outliers_min_{case}.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        print(df.groupby('Year')['Load_sum'].mean())
        # Group by 'Year' and calculate the average 'Total_Load' for each year
        year_average_total_load = df.groupby('Year')['Load_sum'].mean().reset_index(name='average_total_load')

        # Add the region to the DataFrame
        year_average_total_load['region'] = region

        # Append the result to the all_results_df DataFrame
        all_results_df = pd.concat([all_results_df, year_average_total_load], ignore_index=True)


    # Reorder the DataFrame columns if needed
    all_results_df = all_results_df[['region', 'Year', 'average_total_load']]

    # You now have a DataFrame containing the region, Year, and number_of_days for all "min" files
    # Optionally, save this DataFrame to a new CSV file
    all_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_min_outliers_demand_summary_{case}.csv', index=False)







/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_54211/838353845.py:122: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  total_load_df['Total_Load'] = total_load_df.select_dtypes(include=[np.number]).sum(axis=1)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_54211/838353845.py:124: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  total_load_df.reset_index(inplace=True)  # Reset index to turn 'Date' back into a column
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_54211/838353845.py:122: PerformanceWa

Skipping empty file: p103_extreme_demand_outliers_max_projection.csv
Skipping empty file: Utah_extreme_demand_outliers_max_projection.csv
Skipping empty file: p76_extreme_demand_outliers_max_projection.csv
Skipping empty file: p4_extreme_demand_outliers_max_projection.csv
Skipping empty file: p31_extreme_demand_outliers_max_projection.csv
Skipping empty file: Arkansas_extreme_demand_outliers_max_projection.csv
Skipping empty file: p52_extreme_demand_outliers_max_projection.csv
Skipping empty file: p127_extreme_demand_outliers_max_projection.csv
Skipping empty file: p15_extreme_demand_outliers_max_projection.csv
Skipping empty file: p39_extreme_demand_outliers_max_projection.csv
Skipping empty file: Missouri_extreme_demand_outliers_max_projection.csv
Skipping empty file: p130_extreme_demand_outliers_max_projection.csv
Skipping empty file: p69_extreme_demand_outliers_max_projection.csv
Skipping empty file: p45_extreme_demand_outliers_max_projection.csv
Skipping empty file: p84_extreme_de

In [6]:
# There was an issue accessing the file; let's try reading the file again to proceed with the calculations.
import pandas as pd
# Read the state to rb mapping
def string_to_set(string):
    # Remove curly braces and split the string
    elements = string.replace('{', '').replace('}', '').split(',')
    # Remove any extra whitespace and single quotes from each element
    elements = [e.strip().replace("'", "") for e in elements]
    return set(elements)
# Load the dataset
df = pd.read_csv('/Users/ansonkong/Desktop/EIA_loadbystate.csv')


# Assuming 'df' is your DataFrame and it has 'Year' and 'GWh' columns
total_gwh_by_year = df.groupby('year')['GWh'].sum().reset_index()
total_gwh_by_year['Growth Rate'] = total_gwh_by_year['GWh'].pct_change() * 100
total_gwh_by_year[['year','Growth Rate']]
# Calculate the total GWh by State and Year
gwh_by_state_year = df.groupby(['st', 'year'])['GWh'].sum().reset_index()

# Sort the values by State and then Year to ensure the calculation is correct
gwh_by_state_year.sort_values(by=['st', 'year'], inplace=True)

# Calculate the growth rate by state and year
gwh_by_state_year['Growth Rate'] = gwh_by_state_year.groupby('st')['GWh'].pct_change()

# Display the first few rows to verify
print(gwh_by_state_year.head())

# Calculate the mean growth rate for each state
mean_growth_rate_by_state = gwh_by_state_year.groupby('st')['Growth Rate'].mean().reset_index()

state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')
    
# For each state, sum the relevant 'rb' daily sums
for index, row in state_to_ba_mapping.iterrows():
    rb_list = string_to_set(row['reeds_ba_list'])  # Convert string to list/set of rb codes
    state_column = row['state']
    # Ensure only existing columns are summed
    rb_columns = [rb for rb in rb_list]
# State abbreviation mapping
state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Convert full state names to abbreviations in state_to_ba_mapping
state_to_ba_mapping['state'] = state_to_ba_mapping['state'].apply(lambda x: state_abbrev.get(x, x))

# Prepare an empty DataFrame for aggregated RB growth rates
import pandas as pd

# Assuming mean_growth_rate_by_state and state_to_ba_mapping are already loaded
# And state_to_ba_mapping['state'] has been adjusted to use abbreviations as per the state_abbrev mapping

# Preparing an empty list for aggregated RB growth rates
rows = []

# Iterate over each state in mean_growth_rate_by_state
for state_abbr in mean_growth_rate_by_state['st'].unique():
    # Find the RB(s) associated with this state
    rb_set = set()
    for index, row in state_to_ba_mapping.iterrows():
        if state_abbr==row['state']:
            rb_set=string_to_set(row['reeds_ba_list'])  # Assuming there's an 'rb' column indicating the RB
            break
    
    # Calculate mean growth rate for this state (and its associated RBs)
    mean_growth_for_state = mean_growth_rate_by_state[mean_growth_rate_by_state['st'] == state_abbr]['Growth Rate'].mean()
    
    # For each RB associated with the state, add a new row to the list
    # This approach might add multiple rows for the same RB with the same growth rate if multiple states map to it
    for rb in rb_set:
        rows.append({'RB': rb, 'Mean Growth Rate': mean_growth_for_state})

# Convert the list to a DataFrame
rb_growth_rates = pd.DataFrame(rows)

# Since multiple states can map to the same RB, resulting in duplicate RB entries,
# we may need to aggregate these to get a unique mean growth rate per RB
unique_rb_growth_rates = rb_growth_rates.groupby('RB')['Mean Growth Rate'].mean().reset_index()

# Display the DataFrame to verify
print(unique_rb_growth_rates)

# Generate the list of expected RBs from P1 to P134
expected_rbs = [f'p{i}' for i in range(1, 135)]

# Extract the list of RBs from the unique_rb_growth_rates DataFrame
actual_rbs = unique_rb_growth_rates['RB'].unique().tolist()

# Check for missing RBs
missing_rbs = set(expected_rbs) - set(actual_rbs)

# Verify if any RB is missing
if missing_rbs:
    print(f"Missing RBs: {missing_rbs}")
else:
    print("All RBs from P1 to P134 are present.")

# Check for duplicates to ensure uniqueness of RBs in unique_rb_growth_rates
if len(actual_rbs) != unique_rb_growth_rates.shape[0]:
    print("There are duplicate RBs in the DataFrame.")
else:
    print("All RBs in the DataFrame are unique.")


# Calculate the new 'Mean Growth Rate' by raising it to the power of 1/(365*24)
unique_rb_growth_rates['Mean Growth Rate'] = unique_rb_growth_rates['Mean Growth Rate'].apply(lambda x: x/(365*24))

# Display the DataFrame to verify the changes
print(unique_rb_growth_rates)
import pandas as pd
from datetime import datetime
import os
# Your setup
cases = ['projection']
years = range(2020, 2100)
rb_range = range(1, 135)  # Assuming RBs are numbered continuously from P1 to P134

# Reference datetime for calculating the hour difference
reference_datetime = datetime(2010, 1, 1)

# Loop over each combination of case, RB, and year
for case in cases:
    for year in years:
        for rb in rb_range:
            rb_id = f'p{rb}'
            file_path = f'/Volumes/T7/prediction/{case}/{year}/{rb_id}_{year}_mlp_output.csv'
            
            try:
                # Attempt to read the current CSV file
                df = pd.read_csv(file_path)

                # Find the 'Mean Growth Rate' for the current RB from unique_rb_growth_rates DataFrame
                growth_rate = unique_rb_growth_rates.loc[unique_rb_growth_rates['RB'] == rb_id, 'Mean Growth Rate'].iloc[0]
                
                # Convert 'Time_UTC' to datetime and calculate the hour difference from the reference
                df['Time_UTC'] = pd.to_datetime(df['Time_UTC'])
                df['Hour Difference'] = df['Time_UTC'].apply(lambda x: (x - reference_datetime).total_seconds() / 3600)
                
                # Transform 'Load' based on the growth rate and time difference
                df['Load'] = df.apply(lambda row: row['Load'] * (1 + growth_rate) ** row['Hour Difference'], axis=1)
                
                # Define the path to save the transformed DataFrame
                save_path = f'/Volumes/T7/prediction_project/{case}/{year}/{rb_id}_{year}_mlp_output_transformed.csv'
                
                # Create directories if they don't exist
                os.makedirs(os.path.dirname(save_path), exist_ok=True)
                
                # Save the transformed DataFrame
                df.to_csv(save_path, index=False)
                
            except FileNotFoundError:
                print(f'File not found: {file_path}')
                continue



   st  year           GWh  Growth Rate
0  AL  2010  90862.952503          NaN
1  AL  2011  88995.780682    -0.020549
2  AL  2012  86183.388699    -0.031601
3  AL  2013  87854.727290     0.019393
4  AL  2014  90496.828400     0.030074
       RB  Mean Growth Rate
0      p1          0.000572
1     p10          0.004970
2    p100          0.011901
3    p101          0.010893
4    p102          0.010893
..    ...               ...
129   p95          0.002126
130   p96          0.002126
131   p97          0.003322
132   p98          0.003322
133   p99          0.011901

[134 rows x 2 columns]
All RBs from P1 to P134 are present.
All RBs in the DataFrame are unique.
       RB  Mean Growth Rate
0      p1      6.533773e-08
1     p10      5.673265e-07
2    p100      1.358571e-06
3    p101      1.243473e-06
4    p102      1.243473e-06
..    ...               ...
129   p95      2.427114e-07
130   p96      2.427114e-07
131   p97      3.792140e-07
132   p98      3.792140e-07
133   p99      1.358571e

In [12]:
import pandas as pd
import os
from datetime import datetime
for case in ['projection']:
    # Define the directory where the CSV files are located
    directory = f'/Volumes/T7/prediction_project/{case}'

    # Create a list of all 'rb' codes and years
    rb_codes = [f'p{i}' for i in range(1, 135)]
    # Function to read and process files for a given rb code
    def process_rb_files(rb_code):
        rb_df_list = []
        # Loop through all years for the given rb_code
        for year in years:
            # Construct file name
            folder_path = os.path.join(directory, str(year))
            file_name = f"{rb_code}_{year}_mlp_output_transformed.csv"
            file_path = os.path.join(folder_path, file_name)
            # Check if file exists to avoid FileNotFoundError
            if os.path.isfile(file_path):
                # Read the CSV file
                temp_df = pd.read_csv(file_path)
                temp_df=temp_df[['Time_UTC','Load']]
                # Convert 'Time_UTC' to datetime and extract 'Year' and 'Month'
                temp_df['Time_UTC'] = pd.to_datetime(temp_df['Time_UTC'])
                temp_df['Year'] = temp_df['Time_UTC'].dt.year
                temp_df['Month'] = temp_df['Time_UTC'].dt.month
                # Aggregate Load by 'Year' and 'Month'
                aggregated_df = temp_df.groupby(['Year', 'Month'])['Load'].sum().reset_index()
                # Add the aggregated data to the list
                rb_df_list.append(aggregated_df)
    
        # Vertically concatenate all yearly data for the rb code
        concatenated_df = pd.concat(rb_df_list, ignore_index=True)
        return concatenated_df

    years = list(range(2020, 2100))
    months = list(range(1, 13))

    # Create a DataFrame with all combinations of 'Year' and 'Month'
    year_month_df = pd.DataFrame([(y, m) for y in years for m in months], columns=['Year', 'Month'])

    # Placeholder to simulate merging data for multiple rb codes
    final_df = year_month_df.copy()
    for rb_code in rb_codes: 
        rb_df = process_rb_files(rb_code)
        final_df = final_df.merge(rb_df, on=['Year', 'Month'], how='left').rename(columns={'Load': rb_code})
    # Placeholder function to read 'state_to_ba_mapping.csv' and perform aggregation
    def string_to_set(string):
        # Remove curly braces and split the string
        elements = string.replace('{', '').replace('}', '').split(',')
        # Remove any extra whitespace and single quotes from each element
        elements = [e.strip().replace("'", "") for e in elements]
        return set(elements)
    def aggregate_state_data(final_df):
        # Placeholder to simulate reading the 'state_to_ba_mapping.csv' file
        # In practice, this should read the actual file
        state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')

        # Add columns for each state by summing the relevant rb columns based on the mapping
        for index, row in state_to_ba_mapping.iterrows():
            rb_list = string_to_set(row['reeds_ba_list'])
            state_column = row['state']
            # Sum the columns specified in reeds_ba_list for each state
            final_df[state_column] = final_df[list(rb_list)].sum(axis=1)

        # Calculate the total for the USA by summing all state columns
        state_columns = state_to_ba_mapping['state'].values
        final_df['USA'] = final_df[state_columns].sum(axis=1)

        return final_df

    # Apply the placeholder aggregation function to the simulated final_df
    final_aggregated_df = aggregate_state_data(final_df)

    def aggregate_state_data(final_df):
        # Placeholder to simulate reading the 'state_to_ba_mapping.csv' file
        # In practice, this should read the actual file
        state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')

        # Add columns for each state by summing the relevant rb columns based on the mapping
        for index, row in state_to_ba_mapping.iterrows():
            rb_list = string_to_set(row['reeds_ba_list'])
            state_column = row['state']
            # Sum the columns specified in reeds_ba_list for each state
            final_df[state_column] = final_df[list(rb_list)].sum(axis=1)

        # Calculate the total for the USA by summing all state columns
        state_columns = state_to_ba_mapping['state'].values
        final_df['USA'] = final_df[state_columns].sum(axis=1)

        return final_df

    # Apply the placeholder aggregation function to the simulated final_df
    final_aggregated_df = aggregate_state_data(final_df)
    final_aggregated_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/_project_mock_{case}.csv')
    # Define the directory where the CSV files are located
    directory = f'/Volumes/T7/prediction_project/{case}'

    # Create a list of all 'rb' codes and years
    rb_codes = [f'p{i}' for i in range(1, 135)]
    years = list(range(2020, 2100))
    months = list(range(1, 13))


    def process_rb_files_daily(rb_code, years):
        rb_df_list = []
        for year in years:
            folder_path = os.path.join(directory, str(year))
            file_name = f"{rb_code}_{year}_mlp_output_transformed.csv"
            file_path = os.path.join(folder_path, file_name)
            if os.path.isfile(file_path):
                temp_df = pd.read_csv(file_path)[['Time_UTC', 'Load']]
                temp_df['Time_UTC'] = pd.to_datetime(temp_df['Time_UTC'])
                rb_df_list.append(temp_df)
        concatenated_df = pd.concat(rb_df_list, ignore_index=True)
        return concatenated_df
    def aggregate_state_and_usa_daily(final_rb_df):
        # Read the state to rb mapping
        state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')
        
        # For each state, sum the relevant 'rb' daily sums
        for index, row in state_to_ba_mapping.iterrows():
            rb_list = string_to_set(row['reeds_ba_list'])  # Convert string to list/set of rb codes
            state_column = row['state']
            # Ensure only existing columns are summed
            rb_columns = [rb for rb in rb_list if rb in final_rb_df.columns]
            # Summing for each state
            final_rb_df[state_column] = final_rb_df[rb_columns].sum(axis=1)
        
        # Summing all states to get USA total
        state_columns = state_to_ba_mapping['state'].tolist()
        final_rb_df['USA'] = final_rb_df[state_columns].sum(axis=1)
        
        return final_rb_df



    final_dfs = []
    for year in years:
        # Process data for the year (pseudo-code placeholders for actual processing)
        First=True
        for rb_code in rb_codes:
            temp_df = process_rb_files_daily(rb_code, [year])
            if First:
                processed_df=temp_df.copy()
                First=False
                processed_df[rb_code]=processed_df['Load']
                processed_df=processed_df.drop(['Load'], axis=1)
            else:
                # Assuming processed_df and temp_df are defined and have the appropriate columns
                processed_df = processed_df.merge(temp_df, on=['Time_UTC'], how='inner')
                processed_df[rb_code]=processed_df['Load']
                processed_df=processed_df.drop(['Load'], axis=1)
        processed_df = aggregate_state_and_usa_daily(processed_df)
        processed_df['Time_UTC'] = pd.to_datetime(processed_df['Time_UTC'])
        processed_df['Year'] = processed_df['Time_UTC'].dt.year
        processed_df['Month'] = processed_df['Time_UTC'].dt.month


        processed_df=processed_df.drop(['Time_UTC'], axis=1)
        # Group by 'weekday' and 'Year', then apply the aggregations
        # Define aggregations to apply 'max' to all columns except 'Year', 'Month', and 'weekday'
        aggregations = {col: 'max' for col in processed_df.columns if col not in ['Year', 'Month']}

        # Group by 'Year' and 'Month', then aggregate
        grouped_df = processed_df.groupby(['Year', 'Month']).agg(aggregations)



        # Reset index to turn 'Year' and 'Month' back into columns if needed
        grouped_df.reset_index(inplace=True)
        final_dfs.append(grouped_df)


        

    final_df = pd.concat(final_dfs, axis=0, ignore_index=True)
    final_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/_project_max_{case}_monthlly.csv')
    final_df

    import pandas as pd
    import os
    from datetime import datetime
    import numpy as np

    # Define the directory where the CSV files are located
    directory = f'/Volumes/T7/prediction_project/{case}'

    # Create a list of all 'rb' codes and years
    rb_codes = [f'p{i}' for i in range(1, 135)]
    years = list(range(2020, 2100))
    months = list(range(1, 13))


    def process_rb_files_daily(rb_code, years):
        rb_df_list = []
        for year in years:
            folder_path = os.path.join(directory, str(year))
            file_name = f"{rb_code}_{year}_mlp_output_transformed.csv"
            file_path = os.path.join(folder_path, file_name)
            if os.path.isfile(file_path):
                temp_df = pd.read_csv(file_path)[['Time_UTC', 'Load']]
                temp_df['Time_UTC'] = pd.to_datetime(temp_df['Time_UTC'])
                # Additional grouping by day
                temp_df['Day'] = temp_df['Time_UTC'].dt.day
                temp_df['Year'] = temp_df['Time_UTC'].dt.year
                temp_df['Month'] = temp_df['Time_UTC'].dt.month
                aggregated_df = temp_df.groupby(['Year', 'Month', 'Day'])['Load'].sum().reset_index()
                rb_df_list.append(aggregated_df)
        concatenated_df = pd.concat(rb_df_list, ignore_index=True)
        return concatenated_df
    def aggregate_state_and_usa_daily(final_rb_df):
        # Read the state to rb mapping
        state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')
        
        # For each state, sum the relevant 'rb' daily sums
        for index, row in state_to_ba_mapping.iterrows():
            rb_list = string_to_set(row['reeds_ba_list'])  # Convert string to list/set of rb codes
            state_column = row['state']
            # Ensure only existing columns are summed
            rb_columns = [rb for rb in rb_list if rb in final_rb_df.columns]
            # Summing for each state
            final_rb_df[state_column] = final_rb_df[rb_columns].sum(axis=1)
        
        # Summing all states to get USA total
        state_columns = state_to_ba_mapping['state'].tolist()
        final_rb_df['USA'] = final_rb_df[state_columns].sum(axis=1)
        
        return final_rb_df

    def upper_95(x):
        return x.quantile(0.95)

    def lower_5(x):
        return x.quantile(0.05)
    # Define a function to reorganize each column into an array grouped by Year
    def reorganize_into_array(series):
        # Pre-fill the array with NaNs to handle missing weekdays
        array = [np.nan] * 7  # One entry for each day of the week
        for idx, value in series.items():
            # idx is a tuple (Year, weekday), value is the column value
            # Subtract 1 from idx[1] if weekday starts from 1 in your dataset
            array[idx[1]] = value  # Or idx[1] - 1 if weekday is 1-based
        return array

    final_dfs = []
    for year in years:
        # Process data for the year (pseudo-code placeholders for actual processing)
        First=True
        for rb_code in rb_codes:
            temp_df = process_rb_files_daily(rb_code, [year])
            if First:
                processed_df=temp_df.copy()
                First=False
                processed_df[rb_code]=processed_df['Load']
                processed_df=processed_df.drop(['Load'], axis=1)
            else:
                # Assuming processed_df and temp_df are defined and have the appropriate columns
                processed_df = processed_df.merge(temp_df, on=['Year', 'Month', 'Day'], how='inner')
                processed_df[rb_code]=processed_df['Load']
                processed_df=processed_df.drop(['Load'], axis=1)
        processed_df = aggregate_state_and_usa_daily(processed_df)
        processed_df['Date'] = pd.to_datetime(processed_df[['Year', 'Month', 'Day']])

        # Create the 'weekday' column, note that dt.dayofweek returns Monday=0 through Sunday=6, so we add 1
        processed_df['weekday'] = processed_df['Date'].dt.dayofweek 
        processed_df=processed_df.drop(['Month','Day','Date'], axis=1)
        # Group by 'weekday' and 'Year', then apply the aggregations
        aggregations = {col: ['mean','max', upper_95, lower_5] for col in processed_df.columns if col not in ['Year', 'weekday']}
        grouped_df = processed_df.groupby(['Year', 'weekday']).agg(aggregations)
        # Now, flatten the MultiIndex in columns created by aggregation
        grouped_df.columns = ['{}_{}'.format(col[0], col[1]) for col in grouped_df.columns]

        # To further match your requirement, rename the aggregation methods in column names
        grouped_df.rename(columns=lambda x: x.replace('mean', 'mean').replace('upper_95', 'upper').replace('lower_5', 'lower'), inplace=True)

        # Reset index to turn 'Year' and 'weekday' back into columns if needed
        grouped_df= grouped_df.reset_index()
        # First, let's ensure 'weekday' is in the correct data type if not already
        grouped_df['weekday'] = grouped_df['weekday'].astype(int)
        final_dfs.append(grouped_df)
        

    final_df = pd.concat(final_dfs, axis=0, ignore_index=True)
    final_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/_project_mock_{case}_weekly.csv')
    import pandas as pd
    import numpy as np
    import os

    # Define the directory where the CSV files are located and other initial setups
    directory = f'/Volumes/T7/prediction_project/{case}'
    rb_codes = [f'p{i}' for i in range(1, 135)]
    years = list(range(2020, 2100))

    def string_to_set(string):
        elements = string.replace('{', '').replace('}', '').split(',')
        return set(e.strip().replace("'", "") for e in elements)

    def upper_95(x):
        return x.quantile(0.95)

    def lower_5(x):
        return x.quantile(0.05)

    def process_rb_files_hourly(rb_code, year):
        file_name = f"{rb_code}_{year}_mlp_output_transformed.csv"
        file_path = os.path.join(directory, str(year), file_name)
        if os.path.isfile(file_path):
            temp_df = pd.read_csv(file_path, usecols=['Time_UTC', 'Load'])
            temp_df['Time_UTC'] = pd.to_datetime(temp_df['Time_UTC'])
            temp_df[rb_code]=temp_df['Load']
            # temp_df['Hour'] = temp_df['Time_UTC'].dt.hour
            # temp_df['Year'] = temp_df['Time_UTC'].dt.year
            # temp_df['Weekend_or_Weekday'] = np.where(temp_df['Time_UTC'].dt.dayofweek < 5, 'Weekday', 'Weekend')
            # temp_df['Day'] = temp_df['Time_UTC'].dt.day
            # temp_df['Month'] = temp_df['Time_UTC'].dt.month
            temp_df.drop(columns=['Load'], inplace=True)
            return temp_df
        else:
            return pd.DataFrame()

    def aggregate_state_and_usa_daily(df, state_to_ba_mapping):
        for index, row in state_to_ba_mapping.iterrows():
            rb_list = string_to_set(row['reeds_ba_list'])
            state_column = row['state']
            df[state_column] = df[[col for col in df.columns if col in rb_list]].sum(axis=1)
        df['USA'] = df[[row['state'] for index, row in state_to_ba_mapping.iterrows()]].sum(axis=1)
        return df

    def process_yearly_data(year):
        state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')
        First = True

        for rb_code in rb_codes:
            temp_df = process_rb_files_hourly(rb_code, year)
            if First:
                processed_df=temp_df.copy()
                First=False
            else:
                processed_df = processed_df.merge(temp_df, on=['Time_UTC'], how='inner')
        yearly_data=processed_df 

        yearly_data = aggregate_state_and_usa_daily(yearly_data, state_to_ba_mapping)
        yearly_data['Hour'] = yearly_data['Time_UTC'].dt.hour
        yearly_data['Year'] = yearly_data['Time_UTC'].dt.year
        yearly_data['Weekend_or_Weekday'] = np.where(yearly_data['Time_UTC'].dt.dayofweek < 5, 'Weekday', 'Weekend')
        yearly_data.drop(columns=['Time_UTC'], inplace=True)
        columns_to_aggregate = [col for col in yearly_data.columns if col not in ['Hour', 'Year', 'Weekend_or_Weekday']]
        aggregations = {col: ['mean', upper_95, lower_5,'max'] for col in columns_to_aggregate}
            
        # Group by 'Hour', 'Year', 'Weekend_or_Weekday', then aggregate
        grouped_yearly_data = yearly_data.groupby(['Hour', 'Year', 'Weekend_or_Weekday']).agg(aggregations)
        # Flatten the MultiIndex columns if you have multiple columns being aggregated
        grouped_yearly_data.columns = ['_'.join(col).strip() for col in grouped_yearly_data.columns.values]

        # Reset index if you want 'Hour', 'Year', 'Weekend_or_Weekday' back as regular columns
        grouped_yearly_data.reset_index(inplace=True)
        grouped_yearly_data.rename(columns=lambda x: x.replace('mean', 'mean').replace('upper_95', 'upper').replace('lower_5', 'lower'), inplace=True)

        return grouped_yearly_data

    final_dfs = []
    for year in years:
        yearly_data = process_yearly_data(year)
        if not yearly_data.empty:
            final_dfs.append(yearly_data)

    final_df = pd.concat(final_dfs, ignore_index=True)
    final_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/_project_mock_{case}_yearly_aggregated.csv')


KeyboardInterrupt: 

In [8]:
import pandas as pd
import numpy as np
import os
def string_to_set(string):
    # Remove curly braces and split the string
    elements = string.replace('{', '').replace('}', '').split(',')
    # Remove any extra whitespace and single quotes from each element
    elements = [e.strip().replace("'", "") for e in elements]
    return set(elements)

for case in cases:  # Expand cases as needed
    directory = f'/Users/ansonkong/Downloads/Data for nyu work/output/future_{case}_weather'
    ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
    output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier_demand'
    os.makedirs(output_directory, exist_ok=True)

    rb_codes = [f'p{i}' for i in range(1, 135)]
    years = range(2020, 2100)

    for rb_code in rb_codes:
        extreme_outliers_max_list = []
        extreme_outliers_min_list = []

        ci_max_path = os.path.join(ci_directory, f'{rb_code}_CI_max.csv')
        ci_min_path = os.path.join(ci_directory, f'{rb_code}_CI_min.csv')
        
        if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
            ci_max_df = pd.read_csv(ci_max_path)
            ci_min_df = pd.read_csv(ci_min_path)
            
            for year in years:
                weather_file_path = os.path.join(directory, f'{rb_code}_WRF_Hourly_Mean_Meteorology_{year}.csv')
                demand_file_path = f'/Volumes/T7/prediction_project/{case}/{year}/{rb_code}_{year}_mlp_output_transformed.csv'

                if os.path.exists(weather_file_path) and os.path.exists(demand_file_path):
                    df = pd.read_csv(weather_file_path)
                    demand_df = pd.read_csv(demand_file_path)
                    
                    df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                    demand_df['Date'] = pd.to_datetime(demand_df['Time_UTC']).dt.date
                    
                    grouped = df.groupby('Date')['T2'].agg(['max', 'min'])
                    grouped_demand = demand_df.groupby('Date')['Load'].agg(['sum'])  # Changed to sum
                    
                    # Merge to get Load sum values for dates above and below CI thresholds
                    grouped = grouped.merge(grouped_demand, left_index=True, right_index=True, how='left')
                    
                    ci_max_month = ci_max_df[(ci_max_df['rb'] == rb_code)]
                    ci_min_month = ci_min_df[(ci_min_df['rb'] == rb_code)]
                    
                    dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]]
                    dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]]
                    
                    for date, row in dates_above_max_ci.iterrows():
                        extreme_outliers_max_list.append({'rb': rb_code, 'Year': year, 'Date': date, 'Load_sum': row['sum']})
                    
                    for date, row in dates_below_min_ci.iterrows():
                        extreme_outliers_min_list.append({'rb': rb_code, 'Year': year, 'Date': date, 'Load_sum': row['sum']})
        
        # Convert lists to DataFrames
        extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
        extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)
        
        # Save the results
        extreme_outliers_max.to_csv(os.path.join(output_directory, f'{rb_code}_extreme_demand_outliers_max_{case}_project_.csv'), index=False)
        extreme_outliers_min.to_csv(os.path.join(output_directory, f'{rb_code}_extreme_demand_outliers_min_{case}_project_.csv'), index=False)


    # Directories initialization
    directory = f'/Users/ansonkong/Downloads/Data for nyu work/averaged_{case}_weather'
    ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
    output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier_demand'
    os.makedirs(output_directory, exist_ok=True)

    states=['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
            'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
            'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
            'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
            'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming','usa']
    years = range(2020, 2100)

    extreme_outliers_max_list = []
    extreme_outliers_min_list = []

    for state in states:
        extreme_outliers_max_list = []
        extreme_outliers_min_list = []
        ci_max_path = os.path.join(ci_directory, f'{state}_CI_max.csv')
        ci_min_path = os.path.join(ci_directory, f'{state}_CI_min.csv')
        
        if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
            ci_max_df = pd.read_csv(ci_max_path)
            ci_min_df = pd.read_csv(ci_min_path)

            
            for year in years:
                all_rb_dfs = []
                weather_file_path = os.path.join(directory, f'{state}_averaged_weather_{year}.csv')
                if os.path.exists(weather_file_path):
                    if state == 'usa':
                        # If state is USA, set rb_list to include all RB codes from p1 to p134
                        rb_list = [f'p{i}' for i in range(1, 135)]
                    else:
                        state_to_ba_mapping = pd.read_csv('/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/state_to_ba_mapping.csv')
                        state_to_ba_mapping=state_to_ba_mapping[state_to_ba_mapping['state']==state]
                        
                        rb_list = string_to_set(state_to_ba_mapping.iloc[0]['reeds_ba_list'])
                    for rb_code in list(rb_list):
                        demand_file_path = f'/Volumes/T7/prediction_project/{case}/{year}/{rb_code}_{year}_mlp_output_transformed.csv'
                        demand_df = pd.read_csv(demand_file_path)
                    
                        demand_df['Date'] = pd.to_datetime(demand_df['Time_UTC']).dt.date
                        
                        grouped_demand = demand_df.groupby('Date')['Load'].agg(['sum']).rename(columns={'sum': rb_code})  # Rename to allow identification

                        all_rb_dfs.append(grouped_demand)  
                    if all_rb_dfs:
                        total_load_df = pd.concat(all_rb_dfs, axis=1)

                        # Sum across all numerical columns for each date to calculate 'Total_Load'
                        total_load_df['Total_Load'] = total_load_df.select_dtypes(include=[np.number]).sum(axis=1)

                        total_load_df.reset_index(inplace=True)  # Reset index to turn 'Date' back into a column
                        
                        

                    df = pd.read_csv(weather_file_path)
                    df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                    # Find the dates with the highest and lowest T2 for each day
                    grouped = df.groupby('Date')['T2'].agg(['max', 'min'])

                    total_load_df.set_index('Date', inplace=True)

                    # Merge to get Load sum values for dates above and below CI thresholds
                    grouped = grouped.merge(total_load_df, left_index=True, right_index=True, how='left')
                        
                    # Only proceed if the month matches
                    ci_max_month = ci_max_df[ (ci_max_df['State'] == state)]
                    ci_min_month = ci_min_df[ (ci_min_df['State'] == state)]
                        
                    
                    dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]]
                    dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]]

 
                    for date, row in dates_above_max_ci.iterrows():
                        extreme_outliers_max_list.append({'rb': rb_code, 'Year': year, 'Date': date, 'Load_sum': row['Total_Load']})
                    
                    for date, row in dates_below_min_ci.iterrows():
                        extreme_outliers_min_list.append({'rb': rb_code, 'Year': year, 'Date': date, 'Load_sum': row['Total_Load']})

        # Convert lists to DataFrames
        extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
        extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)

        # Save the results
        extreme_outliers_max.to_csv(os.path.join(output_directory, f'{state}_extreme_demand_outliers_max_{case}_project_.csv'), index=False)
        extreme_outliers_min.to_csv(os.path.join(output_directory, f'{state}_extreme_demand_outliers_min_{case}_project_.csv'), index=False)
    # Initialize an empty DataFrame to store all results
    all_results_df = pd.DataFrame()
    folder = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier_demand'

    # List all "max" files in the folder
    max_files = [f for f in os.listdir(folder) if f.endswith(f'_extreme_demand_outliers_max_{case}_project_.csv')]

    for file in max_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_extreme_demand_outliers_max_{case}_project_.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        # Assuming 'Total_Load' is a column in 'df'
        # Group by 'Year' and calculate the average 'Total_Load' for each year
        year_average_total_load = df.groupby('Year')['Load_sum'].mean().reset_index(name='average_total_load')

        # Add the region to the DataFrame
        year_average_total_load['region'] = region

        # Append the result to the all_results_df DataFrame
        all_results_df = pd.concat([all_results_df, year_average_total_load], ignore_index=True)


    # Reorder the DataFrame columns if needed
    all_results_df = all_results_df[['region', 'Year', 'average_total_load']]

    # You now have a DataFrame containing the region, Year, and number_of_days for all "max" files
    # Optionally, save this DataFrame to a new CSV file
    all_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_max_outliers_demand_summary_{case}_project_.csv', index=False)
    all_results_df= pd.DataFrame()


    # List all "min" files in the folder
    min_files = [f for f in os.listdir(folder) if f.endswith(f'_extreme_demand_outliers_min_{case}_project_.csv')]

    for file in min_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_extreme_demand_outliers_min_{case}_project_.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        print(df.groupby('Year')['Load_sum'].mean())
        # Group by 'Year' and calculate the average 'Total_Load' for each year
        year_average_total_load = df.groupby('Year')['Load_sum'].mean().reset_index(name='average_total_load')

        # Add the region to the DataFrame
        year_average_total_load['region'] = region

        # Append the result to the all_results_df DataFrame
        all_results_df = pd.concat([all_results_df, year_average_total_load], ignore_index=True)


    # Reorder the DataFrame columns if needed
    all_results_df = all_results_df[['region', 'Year', 'average_total_load']]

    # You now have a DataFrame containing the region, Year, and number_of_days for all "min" files
    # Optionally, save this DataFrame to a new CSV file
    all_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_min_outliers_demand_summary_{case}_project_.csv', index=False)






/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_54211/1109302568.py:121: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  total_load_df['Total_Load'] = total_load_df.select_dtypes(include=[np.number]).sum(axis=1)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_54211/1109302568.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  total_load_df.reset_index(inplace=True)  # Reset index to turn 'Date' back into a column
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_54211/1109302568.py:121: Performanc

Skipping empty file: Wyoming_extreme_demand_outliers_max_projection_project_.csv
Skipping empty file: Wisconsin_extreme_demand_outliers_max_projection_project_.csv
Skipping empty file: p20_extreme_demand_outliers_max_projection_project_.csv
Skipping empty file: p26_extreme_demand_outliers_max_projection_project_.csv
Skipping empty file: Nebraska_extreme_demand_outliers_max_projection_project_.csv
Skipping empty file: New Mexico_extreme_demand_outliers_max_projection_project_.csv
Skipping empty file: p21_extreme_demand_outliers_max_projection_project_.csv
Skipping empty file: p27_extreme_demand_outliers_max_projection_project_.csv
Skipping empty file: Oregon_extreme_demand_outliers_max_projection_project_.csv
Skipping empty file: p131_extreme_demand_outliers_max_projection_project_.csv
Skipping empty file: p47_extreme_demand_outliers_max_projection_project_.csv
Skipping empty file: Arkansas_extreme_demand_outliers_max_projection_project_.csv
Skipping empty file: p41_extreme_demand_outli

In [9]:
for case in cases:
    # Specify the folder
    # Directories initialization
    directory = f'/Users/ansonkong/Downloads/Data for nyu work/output/future_{case}_weather'
    ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
    output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'
    os.makedirs(output_directory, exist_ok=True)

    rb_codes = [f'p{i}' for i in range(1, 135)]
    years = range(2020, 2100)

    extreme_outliers_max_list = []
    extreme_outliers_min_list = []

    for rb_code in rb_codes:
        extreme_outliers_max_list = []
        extreme_outliers_min_list = []
        ci_max_path = os.path.join(ci_directory, f'{rb_code}_CI_max.csv')
        ci_min_path = os.path.join(ci_directory, f'{rb_code}_CI_min.csv')
        
        if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
            ci_max_df = pd.read_csv(ci_max_path)
            ci_min_df = pd.read_csv(ci_min_path)
            
            for year in years:
                weather_file_path = os.path.join(directory, f'{rb_code}_WRF_Hourly_Mean_Meteorology_{year}.csv')
                if os.path.exists(weather_file_path):
                    df = pd.read_csv(weather_file_path)
                    df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                    df['Month'] = pd.to_datetime(df['Time_UTC']).dt.month
                        
                    # Only proceed if the month matches
                    ci_max_month = ci_max_df[ (ci_max_df['rb'] == rb_code)]
                    ci_min_month = ci_min_df[ (ci_min_df['rb'] == rb_code)]
                        
                    # Find the dates with the highest and lowest T2 for each day
                    grouped = df.groupby('Date')['T2'].agg(['max', 'min'])
                    dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist()
                    # print(ci_max_df)
                    # print(grouped)
                    # print(grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist())
                    for date in dates_above_max_ci:
                        extreme_outliers_max_list.append({'rb': rb_code, 'Year': year, 'Date': date})

                    # Find all dates where min T2 is below the CI_min_lower
                    dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]].index.tolist()
                    for date in dates_below_min_ci:
                        extreme_outliers_min_list.append({'rb': rb_code, 'Year': year, 'Date': date})

        # Convert lists to DataFrames
        extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
        extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)

        # Save the results
        extreme_outliers_max.to_csv(os.path.join(output_directory, f'{rb_code}_extreme_outliers_max_{case}_project_.csv'), index=False)
        extreme_outliers_min.to_csv(os.path.join(output_directory, f'{rb_code}_extreme_outliers_min_{case}_project_.csv'), index=False)


    # Directories initialization
    directory = f'/Users/ansonkong/Downloads/Data for nyu work/averaged_{case}_weather'
    ci_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/CI_results'
    output_directory = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'
    os.makedirs(output_directory, exist_ok=True)

    states=['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
            'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
            'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
            'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
            'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming','usa']
    years = range(2020, 2100)

    extreme_outliers_max_list = []
    extreme_outliers_min_list = []

    for state in states:
        extreme_outliers_max_list = []
        extreme_outliers_min_list = []
        ci_max_path = os.path.join(ci_directory, f'{state}_CI_max.csv')
        ci_min_path = os.path.join(ci_directory, f'{state}_CI_min.csv')
        
        if os.path.exists(ci_max_path) and os.path.exists(ci_min_path):
            ci_max_df = pd.read_csv(ci_max_path)
            ci_min_df = pd.read_csv(ci_min_path)
            
            for year in years:
                weather_file_path = os.path.join(directory, f'{state}_averaged_weather_{year}.csv')
                if os.path.exists(weather_file_path):
                    df = pd.read_csv(weather_file_path)
                    df['Date'] = pd.to_datetime(df['Time_UTC']).dt.date
                    df['Month'] = pd.to_datetime(df['Time_UTC']).dt.month
                        
                    # Only proceed if the month matches
                    ci_max_month = ci_max_df[ (ci_max_df['State'] == state)]
                    ci_min_month = ci_min_df[ (ci_min_df['State'] == state)]
                        
                    # Find the dates with the highest and lowest T2 for each day
                    grouped = df.groupby('Date')['T2'].agg(['max', 'min'])
                    dates_above_max_ci = grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist()
                    # print(ci_max_df)
                    # print(grouped)
                    # print(grouped[grouped['max'] > ci_max_month['CI_max_upper'].iloc[0]].index.tolist())
                    for date in dates_above_max_ci:
                        extreme_outliers_max_list.append({'rb': rb_code, 'Year': year, 'Date': date})

                    # Find all dates where min T2 is below the CI_min_lower
                    dates_below_min_ci = grouped[grouped['min'] < ci_min_month['CI_min_lower'].iloc[0]].index.tolist()
                    for date in dates_below_min_ci:
                        extreme_outliers_min_list.append({'rb': rb_code, 'Year': year, 'Date': date})

        # Convert lists to DataFrames
        extreme_outliers_max = pd.DataFrame(extreme_outliers_max_list)
        extreme_outliers_min = pd.DataFrame(extreme_outliers_min_list)

        # Save the results
        extreme_outliers_max.to_csv(os.path.join(output_directory, f'{state}_extreme_outliers_max_{case}_project_.csv'), index=False)
        extreme_outliers_min.to_csv(os.path.join(output_directory, f'{state}_extreme_outliers_min_{case}_project_.csv'), index=False)


    folder = '/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/outlier'

    # Initialize an empty DataFrame to store all results
    all_results_df = pd.DataFrame()

    # List all "max" files in the folder
    max_files = [f for f in os.listdir(folder) if f.endswith(f'_extreme_outliers_max_{case}_project_.csv')]

    for file in max_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_extreme_outliers_max_{case}_project_.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        # Group by 'Year' and count the number of observations for each year
        year_counts = df.groupby('Year').size().reset_index(name='number_of_days')
        
        # Add the region to the DataFrame
        year_counts['region'] = region
        
        # Append the result to the all_results_df DataFrame
        all_results_df = pd.concat([all_results_df, year_counts], ignore_index=True)

    # Reorder the DataFrame columns if needed
    all_results_df = all_results_df[['region', 'Year', 'number_of_days']]

    # You now have a DataFrame containing the region, Year, and number_of_days for all "max" files
    # Optionally, save this DataFrame to a new CSV file
    all_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_max_outliers_summary_{case}_project_.csv', index=False)


    # Initialize an empty DataFrame to store all results for minimum outliers
    all_min_results_df = pd.DataFrame()

    # List all "min" files in the folder
    min_files = [f for f in os.listdir(folder) if f.endswith(f'_extreme_outliers_min_{case}_project_.csv')]

    for file in min_files:
        # Construct the full file path
        filepath = os.path.join(folder, file)
        
        # Extract the region from the file name
        region = file.split(f'_extreme_outliers_min_{case}_project_.csv')[0]
        
        try:
            # Attempt to read the file into a DataFrame
            df = pd.read_csv(filepath)
        except pd.errors.EmptyDataError:
            print(f"Skipping empty file: {file}")
            continue 
        
        # Group by 'Year' and count the number of observations for each year
        year_counts = df.groupby('Year').size().reset_index(name='number_of_days')
        
        # Add the region to the DataFrame
        year_counts['region'] = region
        
        # Append the result to the all_min_results_df DataFrame
        all_min_results_df = pd.concat([all_min_results_df, year_counts], ignore_index=True)

    # Reorder the DataFrame columns if needed
    all_min_results_df = all_min_results_df[['region', 'Year', 'number_of_days']]

    # You now have a DataFrame containing the region, Year, and number_of_days for all "min" files
    # Optionally, save this DataFrame to a new CSV fileproject
    all_min_results_df.to_csv(f'/Users/ansonkong/Downloads/demand_prediciton_with_weather/resources/all_min_outliers_summary_{case}__.csv', index=False)

Skipping empty file: p5_extreme_outliers_max_projection_project_.csv
Skipping empty file: Texas_extreme_outliers_max_projection_project_.csv
Skipping empty file: Missouri_extreme_outliers_max_projection_project_.csv
Skipping empty file: p134_extreme_outliers_max_projection_project_.csv
Skipping empty file: Montana_extreme_outliers_max_projection_project_.csv
Skipping empty file: p74_extreme_outliers_max_projection_project_.csv
Skipping empty file: p61_extreme_outliers_max_projection_project_.csv
Skipping empty file: p113_extreme_outliers_max_projection_project_.csv
Skipping empty file: p106_extreme_outliers_max_projection_project_.csv
Skipping empty file: p79_extreme_outliers_max_projection_project_.csv
Skipping empty file: p46_extreme_outliers_max_projection_project_.csv
Skipping empty file: Nevada_extreme_outliers_max_projection_project_.csv
Skipping empty file: p53_extreme_outliers_max_projection_project_.csv
Skipping empty file: p8_extreme_outliers_max_projection_project_.csv
Skipp